In [1]:
import numpy as np
import pandas as pd
from math import pi, sqrt

import astropy.table #import tables
from astropy import units as u
from astropy import constants as const

from preprocessing.calc_stellar_params import calc_luminosity, calc_temp
from preprocessing.analyse_errs import classify_err
#from preprocessing.calc_sephi import get_sephi_RM17


In [2]:
# Exoplanet directory:
#exoplanets_dir = "~/Scarlett/OneDrive - Liverpool John Moores University/SEPHI_data/NASA_EA_2022_02_09.csv"
exoplanets_dir = "~/OneDrive/SEPHI_data/NASA_EA_2022_02_09.csv"

# The length of the header in the exoplanets csv file:
header_length = 116

In [3]:
# Read data
exoplanets = pd.read_csv(exoplanets_dir, skiprows=header_length, 
                         usecols=["pl_name", "hostname", "gaia_id", "sy_snum", "sy_pnum", "discoverymethod", "pl_orbper", "pl_orbsmax", "pl_rade", "pl_bmasse", "pl_dens", "pl_orbeccen", "pl_eqt", "pl_orbincl", "st_teff", "st_tefferr1", "st_tefferr2", "st_rad", "st_raderr1", "st_raderr2", "st_mass", "st_met", "st_lum", "st_lumerr1", "st_lumerr2", "st_logg", "st_age", "st_ageerr1", "st_ageerr2", "sy_dist", "sy_plx", "sy_gaiamag"])
# pl_orbper = orbital period [days]
# pl_orbsmax = orbit semi-major axis [au]
#exculuded "st_spectype" due to csv formatting

In [4]:
#print(exoplanets.head())

In [5]:
# Get rid of exoplanets whose star's haven't been observed by Gaia
exoplanets.dropna(subset=["gaia_id"], inplace=True)

In [6]:
# Not many of the exoplanets have the stellar age listed. Working out how many have stellar age:
#count_NaN = exoplanets["st_age"].isna().sum()
#count_value = exoplanets["st_age"].notna().sum()

print("Number of exoplanets with no stellar age listed: ", exoplanets["st_age"].isna().sum())
print("Number of exoplanets with stellar age listed: ", exoplanets["st_age"].notna().sum())

Number of exoplanets with no stellar age listed:  2450
Number of exoplanets with stellar age listed:  2284


In [7]:
# A few don't have stellar effective temperature listed (st_teff):
print("Number of exoplanets with no teff listed: ", exoplanets["st_teff"].isna().sum())
print("Number of exoplanets with teff listed: ", exoplanets["st_teff"].notna().sum())

Number of exoplanets with no teff listed:  227
Number of exoplanets with teff listed:  4507


In [8]:
# Some don't have stellar radius listed
print("Number of exoplanets with no st_rad listed: ", exoplanets["st_rad"].isna().sum())
print("Number of exoplanets with st_rad listed: ", exoplanets["st_rad"].notna().sum())

print("Number of exoplanets with st_raderr1 listed: ", exoplanets["st_raderr1"].notna().sum())
print("Number of exoplanets with st_raderr2 listed: ", exoplanets["st_raderr2"].notna().sum())

Number of exoplanets with no st_rad listed:  320
Number of exoplanets with st_rad listed:  4414
Number of exoplanets with st_raderr1 listed:  4263
Number of exoplanets with st_raderr2 listed:  4240


In [9]:
# Some don't have stellar age listed
print("Number of exoplanets with no st_age listed: ", exoplanets["st_age"].isna().sum())
print("Number of exoplanets with st_age listed: ", exoplanets["st_age"].notna().sum())

Number of exoplanets with no st_age listed:  2450
Number of exoplanets with st_age listed:  2284


In [10]:
# Number of stars with st_rad and st_teff:
count = 0
for i in range(exoplanets.shape[0]):
    if pd.notna(exoplanets["st_teff"].iloc[i]) and pd.notna(exoplanets["st_rad"].iloc[i]):
        count+=1
    else:
        continue

print("Number of exoplanets with st_eff and st_rad listed (can calc L): ", count)

#exoplanets["st_teff"].iloc[i]
    
# Number of stars with st_rad, st_teff and st_age:
count = 0
for i in range(exoplanets.shape[0]):
    if pd.notna(exoplanets["st_teff"].iloc[i]) and pd.notna(exoplanets["st_rad"].iloc[i]) and pd.notna(exoplanets["st_age"].iloc[i]):
        count+=1
    else:
        continue

print("Number of exoplanets with st_teff, st_rad and st_age listed (can calc SEPHI): ", count)

Number of exoplanets with st_eff and st_rad listed (can calc L):  4311
Number of exoplanets with st_teff, st_rad and st_age listed (can calc SEPHI):  2148


In [11]:
# Not many have stellar luminosity listed (but this can be calculated):
print("Number of exoplanets with no L listed: ", exoplanets["st_lum"].isna().sum())
print("Number of exoplanets with L listed: ", exoplanets["st_lum"].notna().sum())

# Do more have their luminosities listed in Gaia?

Number of exoplanets with no L listed:  3825
Number of exoplanets with L listed:  909


In [12]:
print(exoplanets.shape[0])

4734


In [13]:
# Estimate L for stars in 'exoplanets'
luminosities = np.zeros( (exoplanets.shape[0],5) )
for i in range(exoplanets.shape[0]):

    luminosities[i] = calc_luminosity(exoplanets["st_teff"].iloc[i], exoplanets["st_tefferr1"].iloc[i], exoplanets["st_tefferr2"].iloc[i], exoplanets["st_rad"].iloc[i], exoplanets["st_raderr1"].iloc[i], exoplanets["st_raderr2"].iloc[i]) 
    # TODO: If st_lum is listed as nan or if it is a real value with a larger error than the new value then overwrite the previous values
    """
    if pd.isna(exoplanets["st_lum"].iloc[i]) or ( dL1 < exoplanets["st_lumerr1"].iloc[i] and pd.notna(exoplanets["st_lum"].iloc[i]) ): 
        #TODO: overwrite the previous values in exoplanets df
        exoplanets.loc[i,"st_lum"] = L
        exoplanets.loc[i, "st_lumerr1"] = dL1
        exoplanets.loc[i, "st_lumerr2"] = dL2               
        print(L, dL1, dL2)                   
    else:                                                        
        continue  
        """
#print(luminosities)

In [14]:
# Add the luminosities columns to the exoplanets data frame:
exoplanets[ ["calc_L", "calc_Lerr1", "calc_L%err1", "calc_Lerr2", "calc_L%err2"] ] = luminosities
#print(exoplanets)

In [15]:
# Array to store classification:
classes_L = np.zeros( (exoplanets.shape[0]), dtype=int )
#print(classes_L[0:40])

# Classify the luminosity uncertainties using the function:
for i in range(exoplanets.shape[0]):
    classes_L[i] = classify_err(exoplanets["st_lum"].iloc[i], exoplanets["st_lumerr1"].iloc[i], exoplanets["st_lumerr2"].iloc[i], exoplanets["calc_L%err1"].iloc[i], exoplanets["calc_L%err2"].iloc[i])
                                                                                                             
print(classes_L[0:100])
print(len(classes_L))

[1 2 2 2 2 1 3 1 1 1 1 1 1 1 3 3 3 1 3 2 2 2 1 1 1 1 1 2 2 2 2 2 3 1 1 1 1
 1 3 2 2 2 2 3 2 2 1 2 1 2 2 2 2 2 3 2 1 1 3 2 3 3 1 1 2 3 2 1 1 2 1 1 3 3
 1 3 2 2 2 2 2 2 2 2 2 1 2 3 3 2 2 2 2 2 2 2 2 2 2 2]
4734


In [16]:
print("Number of exoplanets with errors in st_lum < calc_L or no dcalc_L: ", np.count_nonzero(classes_L == 1),"\nNB: there are 909 planets with st_lum listed.") 
print("Number of exoplanets with errors in calc_L < st_lum or no dst_lum: ", np.count_nonzero(classes_L == 2))

Number of exoplanets with errors in st_lum < calc_L or no dcalc_L:  646 
NB: there are 909 planets with st_lum listed.
Number of exoplanets with errors in calc_L < st_lum or no dst_lum:  3574


In [17]:
# Add the crror class column to the exoplanets data frame
exoplanets[ "Lerr_class" ] = classes_L
#print(exoplanets)

In [18]:
# Estimate T for stars in 'exoplanets'
temps = np.zeros( (exoplanets.shape[0], 5) )
for i in range(exoplanets.shape[0]):

    temps[i] = calc_temp(exoplanets["st_rad"].iloc[i], exoplanets["st_raderr1"].iloc[i], exoplanets["st_raderr2"].iloc[i], exoplanets["st_lum"].iloc[i], exoplanets["st_lumerr1"].iloc[i], exoplanets["st_lumerr2"].iloc[i]) 

#print(temps)

In [19]:
# Add calc_teff to exoplanets df:
exoplanets[ ["calc_T", "calc_Terr1", "calc_T%err1", "calc_Terr2", "calc_T%err2"] ] = temps
#print(exoplanets.head(10))
# Temps and errors look good!

In [20]:
# Compare the uncertainties in st_teff and calc_teff: 
# Give a classification according to which error is lower:
# 0 (equal), 1 (st_lum_combined is 'best'), 2 (calc_L_combined is 'best'), 3 (no classification, neither have both +ve and -ve errors avaiable)

# Array to store classification:
classes_T = np.zeros( (exoplanets.shape[0]), dtype=int )

for i in range(exoplanets.shape[0]):
    classes_T[i] = classify_err(exoplanets["st_teff"].iloc[i], exoplanets["st_tefferr1"].iloc[i], exoplanets["st_tefferr2"].iloc[i], exoplanets["calc_T%err1"].iloc[i], exoplanets["calc_T%err2"].iloc[i])
                                                                                                             
print(classes_T[0:100])
print(len(classes_T))

[1 1 1 1 1 1 1 1 1 2 2 2 1 1 3 3 3 1 1 1 1 1 2 2 2 3 2 1 1 1 1 1 3 2 2 2 2
 2 3 1 1 1 1 3 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 2 3 3
 2 3 1 1 1 1 1 1 1 1 1 1 1 3 3 1 1 1 1 1 1 1 1 1 1 1]
4734


In [21]:
# Counting how many st_teff have smaller error and how many temp have smaller error
print("Number of exoplanets with errors in st_teff < calc_teff or incomplete errors on calc_teff: ", np.count_nonzero(classes_T == 1),"\nNB: there are 4507 planets with st_teff listed.") 
print("Number of exoplanets with errors in calc_T < st_teff or incomplete errors on st_teff: ", np.count_nonzero(classes_T == 2))

Number of exoplanets with errors in st_teff < calc_teff or incomplete errors on calc_teff:  4103 
NB: there are 4507 planets with st_teff listed.
Number of exoplanets with errors in calc_T < st_teff or incomplete errors on st_teff:  272


In [22]:
# Adding the classes_T array to the exoplanets df
exoplanets["Terr_class"] = classes_T
print(exoplanets)

         pl_name  hostname                       gaia_id  sy_snum  sy_pnum  \
0       11 Com b    11 Com  Gaia DR2 3946945413106333696        2        1   
1       11 UMi b    11 UMi  Gaia DR2 1696798367260229376        1        1   
2       14 And b    14 And  Gaia DR2 1920113512486282240        1        1   
3       14 Her b    14 Her  Gaia DR2 1385293808145621504        1        2   
4     16 Cyg B b  16 Cyg B  Gaia DR2 2135550755683407232        3        1   
...          ...       ...                           ...      ...      ...   
4909   ups And b   ups And   Gaia DR2 348020448377061376        2        3   
4910   ups And c   ups And   Gaia DR2 348020448377061376        2        3   
4911   ups And d   ups And   Gaia DR2 348020448377061376        2        3   
4912   ups Leo b   ups Leo  Gaia DR2 3794167001116433152        1        1   
4913    xi Aql b    xi Aql  Gaia DR2 4298361114750843904        1        1   

      discoverymethod    pl_orbper  pl_orbsmax  pl_rade   pl_bm

In [23]:
#exoplanets_tab = astropy.table.Table.from_pandas(exoplanets)
#exoplanets["pl_rade"].unit = astropy.units.earthRad
#print(exoplanets_tab)

In [24]:
# TODO: I can drop the two percentage errors on L and T

In [25]:
'''
# convert the inputs add units
pl_mass = exoplanets["pl_bmasse"].iloc[13] * u.earthMass
pl_rad = exoplanets["pl_rade"].iloc[13] * u.earthRad
pl_a = exoplanets["pl_orbsmax"].iloc[13] * u.AU
teff = exoplanets["st_teff"].iloc[13] * u.K
lum = exoplanets["st_lum"].iloc[13] * u.dex(u.L_sun)
age = exoplanets["st_age"].iloc[13] * 10**9 * u.yr
print("planet mass, radius and semi-major axis: ", pl_mass, pl_rad, pl_ a)
'''

'\n# convert the inputs add units\npl_mass = exoplanets["pl_bmasse"].iloc[13] * u.earthMass\npl_rad = exoplanets["pl_rade"].iloc[13] * u.earthRad\npl_a = exoplanets["pl_orbsmax"].iloc[13] * u.AU\nteff = exoplanets["st_teff"].iloc[13] * u.K\nlum = exoplanets["st_lum"].iloc[13] * u.dex(u.L_sun)\nage = exoplanets["st_age"].iloc[13] * 10**9 * u.yr\nprint("planet mass, radius and semi-major axis: ", pl_mass, pl_rad, pl_ a)\n'

In [26]:
# I could then merge the exoplanets data with the phase-space densities calculated from edr3.
# I would need to use a 

In [27]:
def likelihood_telluric(planet_mass, planet_radius, verbose):
    
    """
    planet_mass: units are converted to Earth masses
    planet_radius: units are converted to Earth radii
    verbose: 
    """
    
    if(verbose):
        print("Composition")
    
    # Determining radius of a 100% MgSiO3 planet of the same mass
    # Using result of 3rd-order polynomial fit to Table 1 in Zeng & Sasselov 2013
    log10_MgSiO3_radius = np.poly1d( np.array([-0.0066252 , -0.02274424,  0.30285182,  0.02052977]))
    MgSiO3_radius = 10.**(log10_MgSiO3_radius(planet_mass.to(u.Mearth).value))
    #print("MgSi03 radius: ", MgSiO3_radius)
   
    # Determining radius of a 50% MgSiO3, 50% H2O planet of the same mass
    # Using result of 3rd-order polynomial fit to Table 1 in Zeng & Sasselov 2013
    log10_MgSiO3_H2O_radius = np.poly1d( np.array([-0.00637393, -0.01837899,  0.28980072,  0.10391018]))
    MgSiO3_H2O_radius = 10.**(log10_MgSiO3_H2O_radius(planet_mass.to(u.Mearth).value)) # converted to Earth masses

    sigma = (MgSiO3_H2O_radius - MgSiO3_radius) / 3.0
    
    #--------------------------
    # Calculating likelihood

    c1 = np.where(planet_radius.to(u.Rearth).value <= MgSiO3_radius) #condition 1
    c2 = np.where(np.logical_and( planet_radius.to(u.Rearth).value > MgSiO3_radius, \
                                  planet_radius.to(u.Rearth).value <= MgSiO3_H2O_radius) ) #condition 2
    
    likelihood = np.zeros(planet_mass.size)  #initialising with zero likelihood   
    if(c1[0].size>0):
        likelihood[c1] = 1.0
        #print("c1: ", c1)
    if(c2[0].size>0):
        likelihood[c2] = np.exp( -0.5 * ( (planet_radius.to(u.Rearth).value[c2] - MgSiO3_radius[c2])/sigma[c2] )**2 )
        #print("c2: ", c2)
    
    if(verbose):
        print("Sim 1: (Telluric planet) = ", likelihood, "\n")

    return likelihood

In [28]:
#Function to determine likelihood that planet has an atmosphere.
def likelihood_atmosphere(planet_mass, planet_radius, verbose):
    
    if(verbose):
        print("Normalised physical characteristics")
    
    #calculating the relative escape velocity of planet wrt Earth
    v_e = (planet_mass.to(u.kg) * const.R_earth / (const.M_earth * planet_radius.to(u.m)) )**0.5
    if(verbose):
        print ("escape velocity relative to Earth",v_e)
        
    # calculating likelihood    
    c1 = np.where(v_e.value < 1.0)  #condition 1
    #print("c1: ", c1)
    c2 = np.where(v_e.value >= 1.0) #condition 2
    #print("c2: ", c2)

    likelihood = np.zeros(planet_mass.size)  #initialising with zero likelihood  
    #print(likelihood)
    if(c1[0].size>0):
        likelihood[c1] = np.exp(-0.5*(3.*(v_e.value[c1]-1.))**2)
    if(c2[0].size>0):
        likelihood[c2] = np.exp(-0.5*(3.*(v_e.value[c2]-1.)/7.66)**2) # TODO: stopping at this line. Index to scalar
    
    if(verbose):
        print("Sim 2: (Atmosphere) = ", likelihood, "\n")
    return likelihood

In [29]:
#Function to determine likelihood that planet has an liquid water on surface.
#Uses models in Kopparapu+13,ApJ,765,131 and Kopparapu+14,ApJL,787,29
#Note: only works for stars with effective stellar temperatures of 2600K < T_eff <7200K,
#corresponding to F,G,K,M stars.
# SNL 3/8: At the moment the function deals 'correctly' with remnants, in that if 
# L_star = 0 and T_eff_star = nan, the function return a likelihood value = '0.'
# However, the code produces warnings and errors, so in the future it would be 
# better to deal with this more elegantly. 
def likelihood_surface_liquid_water(T_eff_star, L_star, planet_mass, planet_semi_major_axis, verbose):

    if(verbose):
        print("Habitability")
    
    possible_water = (L_star > 0.) & ~np.isinf(planet_semi_major_axis) # where the host star has non-zero luminosity and the planet is not free-floating

    #Making sure the stellar effective temperature is in the correct range
    if( np.any(T_eff_star[possible_water].value < 2600.) or np.any(T_eff_star[possible_water].value > 7200.) ):
        # print("Warning: stellar effective temperature outside correct range of 2600K < T_eff <7200K. Truncating.")
        T_eff_star.value[np.where( np.logical_and(T_eff_star.value<2700., possible_water) )] = 2700.0 
        T_eff_star.value[np.where( np.logical_and(T_eff_star.value>7200., possible_water) )] = 7200.0 
        # return -1.0
    
    #Coefficient updates from Table 1 of Kopparapu+14,ApJL,787,29
    #These are similar to Kopparapu+13,ApJ,765,131 but the runaway greenhouse depends on planet mass.
    #To get coefficients: download HZs.f90 from Kop+14 electronic file associated with the paper, 
    #then compile and run that to generate HZs.dat and HZ_coefficients.dat. 
    #Coeffs copied here to avoid speed penalty in opening/reading from file
    ##numbers: 1=recent venus,2=runaway greenhouse,3=max. greenhouse,4=early mars,5=run. ghouse (5Mearth), 6=run. ghouse (0.1Mearth)
    
    S_eff_sol=np.repeat(np.array([[1.77600E+00,1.10700E+00,3.56000E-01,3.20000E-01,1.18800E+00,9.90000E-01]]), T_eff_star.size, axis=0)
    a=np.array([[2.13600E-04,1.33200E-04,6.17100E-05,5.54700E-05,1.43300E-04,1.20900E-04]])
    b=np.array([[2.53300E-08,1.58000E-08,1.69800E-09,1.52600E-09,1.70700E-08,1.40400E-08]])
    c=np.array([[-1.33200E-11,-8.30800E-12,-3.19800E-12,-2.87400E-12,-8.96800E-12,-7.41800E-12]])
    d=np.array([[-3.09700E-15,-1.93100E-15,-5.57500E-16,-5.01100E-16,-2.08400E-15,-1.71300E-15]])
    
    # From Kopparapu. 5780K is the surface temp for sun They see how much cooler or hotter than the sun the star is
    T_star_K = T_eff_star - 5780*u.K
    T_star = np.reshape(T_star_K.value, (1,T_star_K.size))
  
    # Calculating the distance from the star for each of the boundaries. First the mass-independent ones
    S_eff = S_eff_sol.T + np.dot(a.T,T_star) + np.dot(b.T,T_star**2) + np.dot(c.T,T_star**3) + np.dot(d.T,T_star**4)
    D = (L_star.to(u.W)/const.L_sun/S_eff)**0.5 * u.au

    #-------------------
    # Calculating distance from star at which planet undergoes runaway greenhouse. In Kopparapu+14 this is planet-mass-dependent
    # SNL: note the mass boundaries where the switch between different regimes occurs are not defined in SEPHI paper.
    #      I've made the boundary roughly mid-range between the discrete masses but this may vary from the online calculator.
 
    c1 = np.where( (planet_mass.value<0.5) & possible_water)
    c2 = np.where( (planet_mass.value>2.) & possible_water)
    c3 = np.where( np.logical_and(planet_mass.value>=0.5, planet_mass.value <=2.) & possible_water)
    
    D_RG = np.zeros(planet_mass.size)*u.au
    if(c1[0].size>0):
        D_RG[c1] = D[5,c1]; #print (c1[0].size, "Planet mass <0.5Msun")
    if(c2[0].size>0):
        D_RG[c2] = D[4,c2]; #print (c2[0].size, "Planet mass >2Msun")
    if(c3[0].size>0):
        D_RG[c3] = D[1,c3]; #print (c3[0].size, "Planet mass 0.5 - 2 Msun")
    
    # Checking distances make sense, i.e. D1<D_RG<D3<D4
    # Note, these distances get screwed up if the temperature is outside the 2700K < T < 7200K range
    # if not ( np.all(D[0,:]<D_RG) and np.all(D_RG<D[2,:]) and np.all(D[2,:]<D[3,:])):
    #     print("Distance calculations incorrect, because temperature outside correct range of 2600K < T_eff <7200K")
        #return -1.
 
    if(verbose):
        print("Green Zone Inner Radius = ", D_RG)
    if(verbose):
        print("Green Zone Outer Radius = ", D[2,:])
    if(verbose):
        print("Incident Effective Flux = ", S_eff)
    
    #-------------------
    # Determining the likelihood
        
    likelihood = np.zeros(planet_mass.size)  #initialising with zero likelihood 
        
    c4 = np.where( (planet_semi_major_axis < D_RG) & possible_water)
    c5 = np.where( np.logical_and(planet_semi_major_axis >= D_RG, planet_semi_major_axis <= D[2,:]) & possible_water)
    c6 = np.where( (planet_semi_major_axis > D[2,:]) & possible_water)
 
    if(c4[0].size>0): 
        sigma_31=(D_RG-D[0,:])/3.0   
        likelihood[c4]=np.exp(-0.5*((planet_semi_major_axis[c4] - D_RG[c4])/sigma_31[c4])**2)
        if(verbose):
            print("Zone = Hot")
        
    if(c5[0].size>0): 
        likelihood[c5] = 1.0
        if(verbose):
            print("Zone = Green")
    
    if(c6[0].size>0): 
        sigma_32=(D[3,:]-D[2,:])/3.0 
        likelihood[c6]=np.exp(-0.5*((planet_semi_major_axis[c6] - D[2,c6])/sigma_32[c6])**2)
        if(verbose):
            print("Zone = Cold")
 
    if(verbose):
        print("Sim 3: (Liquid Water) = ", likelihood, "\n")

    return likelihood

In [36]:
#Function to determine the likelihood that a planet has a magnetic moment similar to the Earth.
def likelihood_magnetic_moment(stellar_mass, planet_semi_major_axis, planet_system_age, planet_radius, planet_mass, verbose):
    
    if(verbose):
        print("Normalised Magnetic Field")
    
    #---------------------------
    #Pre-amble preparation
    
    #Adding mass and radius of Neptune and Jupiter which are needed in defining planet types
    neptune_radius = 24622000 * u.m; neptune_mass   = 1.02413E26 * u.kg
    jupiter_radius = 71492000 * u.m; jupiter_mass   = 1.8982E27 * u.kg
    
    #calculating density ratio of planet with solar system objects as this is needed several times
    planet_density_ratio_Earth   = (planet_mass.to(u.kg) * const.R_earth**3/planet_radius.to(u.m)**3/const.M_earth)
    planet_density_ratio_Jupiter = (planet_mass.to(u.kg) * jupiter_radius **3/planet_radius.to(u.m)**3/jupiter_mass)
    planet_density_ratio_Neptune = (planet_mass.to(u.kg) * neptune_radius **3/planet_radius.to(u.m)**3/neptune_mass)
    
    #---------------------------
    #Determining whether the planet tidally locked. Using Eq 1 from 
    #Griessmeier+09, Icarus,199,526 to determine how long it takes a planet of given 
    #structure and radius from host star of given mass to become tidally locked.
    #There are *many* assumptions and simplifications that go into this.
    #However, from the discussion in G+09, many of the factors are not expected to vary 
    #by more than factors of a few for reasonable exoplanet properties.
    #The tidal locking timescale is dominated primarily by the star-planet distance (^6) and 
    #secondly the star-planet mass ratio (^2)
    #3 fiducial cases: 1. Earth, 2."small & big super Earth" (6M_E, 1.63R_E & 10M_E, 1.86R_E), 3. "Ocean planet" (6M_E, 2R_E) 
    
    #Determining factors of order unity
    alpha_G07 = 1./3. #structure of planet. Assume planet structure similar to Earth.
    omega_final = 0.0 / u.s #ignore final angular velocity of the planet ("can be neglected...for the planets of interest in this work")
    #omega_initial_upper = 1.8 #upper value of initial angular velocity (based on early Earth-moon system, day length = 13.1 hours).
    #omega_initial_lower = 0.8 #lower value of initial angular velocity (day length = 30 hours).
    omega_initial = 1.0 / u.s #SNL: taking rough average between upper and lower values. Can therefore ignore from tau_synch calculation below.

    #Calculating the tidal dissipation factor
    k_2_p = 0.3     #Apparently this is suitable for Earth, "small super Earth", and "ocean planet" cases (need to check relevance of those for our work) 
    Q_P_Earth = 12  #planetary tidal dissipation factor for Earth (was larger in the past but Earth's shallow sees dissipataed energy. Was high when single continent)
    Q_P_other = 100 #planetary tidal dissipation factor for both the super Earths and Ocean planet
    Q_P_prime_Earth = (3. * Q_P_Earth)/(2.*k_2_p) #modified Q value for Earth
    Q_P_prime_other = (3. * Q_P_other)/(2.*k_2_p) #modified Q value for super Earths and ocean world
    
    #First step of calculating tidal synchronisation time (how long till a planet gets tidally locked)
    tau_synch = (4./9.) * alpha_G07 * (planet_radius.to(u.m)**3/const.G/planet_mass.to(u.kg)) \
                 * (omega_initial - omega_final) \
                 * (planet_mass.to(u.kg)/stellar_mass.to(u.kg))**2 \
                 * (planet_semi_major_axis.to(u.m)/planet_radius.to(u.m))**6
    #print("tau_synch: ", tau_synch)
    
    #Next step: determining whether to use Earth-like, or "other" (super-Earth/Ocean) tidal dissipation factor
    c1 = np.where(  np.logical_and(planet_mass.to(u.kg) < 2.0 * const.M_earth, planet_radius.to(u.m) < 1.5 * const.R_earth))
    c2 = np.where(~(np.logical_and(planet_mass.to(u.kg) < 2.0 * const.M_earth, planet_radius.to(u.m) < 1.5 * const.R_earth)))
  
    if(c1[0].size>0):
        tau_synch[c1] *=  Q_P_prime_Earth
    if(c2[0].size>0):
        tau_synch[c2] *=  Q_P_prime_other

    #------------------------------
    #Now determining the magnetic moment ratio between the planet and Earth that
    #is used in the likelihood calculation. The ratio depends on whether the 
    #planet is tidally locked, and how its radius and density compare to 
    #terrestrial-like, ice giant and gas giant planets.
    
    # Determining conditions if planets tidally locked and density ranges wrt solar system planets
    #print("tau_synch2: ", tau_synch)
    #print("planet_system_age2 :", planet_system_age)
    tidally_locked     = tau_synch.to(u.Gyr) < planet_system_age.to(u.Gyr) #TODO: here is is trying to convert planet_system_age from solMass to Gyr
    earth_like_density = planet_density_ratio_Earth >= 1.0
    ice_giant_density  = np.logical_and(planet_density_ratio_Earth < 1.0, planet_density_ratio_Earth > 0.18)
    neptune_density    = np.logical_and(planet_density_ratio_Earth <=0.18, planet_density_ratio_Earth >0.16) 
    jupiter_density    = planet_density_ratio_Earth <=0.16 
    
    c3 = np.where(tidally_locked)
    c4 = np.where(np.logical_and(~tidally_locked, earth_like_density))
    c5 = np.where(np.logical_and(~tidally_locked, ice_giant_density))
    c6 = np.where(np.logical_and(~tidally_locked, neptune_density))
    c7 = np.where(np.logical_and(~tidally_locked, jupiter_density))
        
    mag_moment_ratio = np.zeros(planet_mass.size) #initialising magnetic moment ratio array
  
    # ---------------------------
    # Planet tidally locked: use Eq7 in Rodriguez-Mozoz & Moya (2017) (originally from Sano93)
    if(c3[0].size>0):
        
        if(verbose):
            print("Planet rotation = locked")
        
        #SNL: need to assume something about angular frequency ratio of planet with that of Earth. Assume omega ratio = 1.
        omega_ratio = 1.0
        mag_moment_ratio[c3] = planet_density_ratio_Earth[c3]**0.5 \
                                 * (planet_radius.to(u.m)[c3]/const.R_earth)**(7./2.) \
                                     * omega_ratio 
                                     
    else: 
        if(verbose):
            print("Planet rotation = Free")  
                               
    # ---------------------------
    # Planet not tidally locked: use Eq 9 & 10 and Table 3 from Rodriguez-Mozoz & Moya (2017)     
    
    # Calculations below require knowing nature of dynamo (multipolar, internally heated, dipolar) to determine alpha_SOC: 
    #  - based on Olson & Christensen, 2006, Earth & Planet Science Letters, 250, 561
    #  - These required obs pars are difficult/unknown even for SS objects -> now way can know these for exoplanets
    #  - Pragmatic solution = use the discussion in section 6 and Figure 7 to determine a planet's SOC by similarity with SS objects
    #       - Earth & Jupiter like = bipolor (alpha_SOC=1.0)
    #       - Neptune-like & between Earth and ice giant = internally heated dynamo (alpha_SOC=0.15)
                       
    if(c4[0].size>0):                                  # Earth-like  
        if(verbose):
            print("Planet density = Earth-like \nEstimated regime = Dipolar") 
        alpha_SOC = 1.0 # dipolar        
        beta_1 = planet_radius.to(u.m)/const.R_earth
        mag_moment_ratio[c4] = alpha_SOC * beta_1[c4]**(10./3.) * beta_1[c4]**(1./3.)
        
    
    if(c5[0].size>0):                                 # Between telluric and ice giant
        if(verbose):
            print("Planet density = Between telluric and ice giant \nEstimated regime = Internally heated dynamo") 
        alpha_SOC = 0.15 # internally heated dynamo
        beta_1 = planet_radius.to(u.m)/const.R_earth
        mag_moment_ratio[c5] = alpha_SOC * 0.45**0.5 * (1.8 * beta_1[c5])**(10./3.) * (4.*beta_1[c5])**(1./3.)
               
    if(c6[0].size>0):                                 # Neptune-like ice giant
        if(verbose):
            print("Planet density = Neptune-like \nEstimated regime = Internally heated dynamo") 
        alpha_SOC = 0.15 # internally heated dynamo
        beta_1 = planet_radius.to(u.m)/neptune_radius
        mag_moment_ratio[c6] = alpha_SOC * 0.18**0.5 * (4.5 * beta_1[c6])**(10./3.) * (20.*beta_1[c6])**(1./3.)
   
    if(c7[0].size>0):                                 # Jupiter-like gas giant
        if(verbose):
            print("Planet density = Jupiter-like \nEstimated regime = Dipolar") 
        alpha_SOC = 1.0 # dipolar 
        beta_1 = planet_radius.to(u.m)/jupiter_radius
        beta_2 = planet_density_ratio_Jupiter
        mag_moment_ratio[c7] = alpha_SOC * 0.16**0.5 * (16.*beta_1[c7]*beta_2[c7])**(10./3.) * (100.*beta_1[c7]*beta_2[c7])**(1./3.)
  
    #-------------------------------
    #Finally, calculating and returning likelihood
    #If magnetic moment ratio > 1.0, planet has stronger magnetic field than Earth so is protected: likelihood = 1.0.
    #If magnetic moment ratio < 1.0, planet has weaker magnetic field, so likelihood drops         
    
    if(verbose):
        print("magnetic moment ratio = ", mag_moment_ratio)
    c8 = np.where(mag_moment_ratio < 1.0)    
        
    likelihood = np.ones(planet_mass.size)
    if(c8[0].size>0):
        likelihood[c8] = np.exp(-0.5*( 3.*(mag_moment_ratio[c8]-1.0))**2 ) 

    if(verbose):
        print("Sim 4: (Magnetic Field) = ", likelihood, "\n")

    return likelihood

In [31]:
def get_sephi_RM17(planet_mass, planet_radius, planet_semi_major_axis, T_eff_star, L_star, stellar_mass, planet_system_age, verbose=False):
    """
    The inputs should be the already collated 'best' values'
    planet_mass: array of planet masses. Any units, with units included in array
    planet_radius: array of planet radii. Any units, with units included in array
    planet_semi_major_axis: array, any distance units
    T_eff_star: array, any temp units
    L_star: array, any units
    stelalr_mass: array, any mass units
    
    NB: this function assumes that the arrays are the same length and that indicies correspond to prioperties of the same planet
    """
    
    # Condition 1: stores the indicies where at least one of the necessary planet/stellar properties is NaN
    # TODO: wheck whether this works with multiple NaN properties
    c1 = np.where(np.isnan(planet_mass.value) | np.isnan(planet_radius.value) | np.isnan(planet_semi_major_axis.value) | np.isnan(T_eff_star.value) | np.isnan(L_star.value) | np.isnan(planet_system_age.value) )
    #print("c1: ", c1)
    
    # Condition 2: stores the indicies where all planet/stellar properties are available
    c2 = np.where(np.isfinite(planet_mass.value) & np.isfinite(planet_radius.value) & np.isfinite(planet_semi_major_axis.value) & np.isfinite(T_eff_star.value) & np.isfinite(L_star.value) & np.isfinite(planet_system_age.value) )
    #print("c2: ", c2)
    
    # Empty arrays to store each likelihood:
    likelihood_1 = np.zeros(planet_mass.size) # Empty array to store likelihood telluric
    #print("Likelihood 1 shape: ", likelihood_1.shape)
    #print("Likelihood 1: ", likelihood_1)
    likelihood_2 = np.zeros(planet_mass.size) # Empty array to store likelihood atmosphere
    likelihood_3 = np.zeros(planet_mass.size) # Empty array to store likelihood surface liquid water
    likelihood_4 = np.zeros(planet_mass.size) # Empty array to store likelihood magnetic moment
    
    # Empty array to store the combined likelihoods (SEPHIs):
    combined = np.zeros(planet_mass.size) 
    
    # If there is at least one NaN value, then the likelihoods and combined likelihood is NaN; SEPHI cannot be calculated when parameters are missing:
    if(c1[0].size>0):
        likelihood_1[c1] = np.nan
        likelihood_2[c1] = np.nan
        likelihood_3[c1] = np.nan
        likelihood_4[c1] = np.nan
        combined[c1] = np.nan
        
    # If there is at least one planet with all planetary/stellar parameters available, calculate each likelihood and the combined likelihood (SEPHI):
    if(c2[0].size>0):
        #Determine likelihoods at 4 different stages:
        likelihood_1[c2] = likelihood_telluric(planet_mass[c2], planet_radius[c2], verbose)
        likelihood_2[c2] = likelihood_atmosphere(planet_mass[c2], planet_radius[c2], verbose)
        likelihood_3[c2] = likelihood_surface_liquid_water(T_eff_star[c2], L_star[c2], planet_mass[c2],  planet_semi_major_axis[c2], verbose)
        likelihood_4[c2] = likelihood_magnetic_moment(stellar_mass[c2], planet_semi_major_axis[c2], planet_system_age[c2], planet_radius[c2], planet_mass[c2], verbose)
        # Determine SEPHI as geometric mean of the 4 different likelihoods:
        combined[c2] = (likelihood_1[c2] * likelihood_2[c2] * likelihood_3[c2] * likelihood_4[c2])**(1./4.) 

    if(verbose):
        print("Combined likelihood: = ", combined, "\n" )
        
    return combined

In [32]:
pl_mass = exoplanets["pl_bmasse"].to_numpy() * u.earthMass
pl_rad = exoplanets["pl_rade"].to_numpy() * u.earthRad
pl_a = exoplanets["pl_orbsmax"].to_numpy() * u.AU
teff = exoplanets["st_teff"].to_numpy() * u.K
lum = exoplanets["st_lum"].to_numpy() * u.dex(u.L_sun)
st_mass = exoplanets["st_mass"].to_numpy() * u.solMass
age = exoplanets["st_age"].to_numpy() * u.Gyr
print("planet mass: ", pl_mass, "\n data shape: ", pl_mass.shape)
      #, pl_rad, pl_ a)

planet mass:  [6165.6     4684.8142  1525.5     ... 1313.22     162.09249  890.     ] earthMass 
 data shape:  (4734,)


In [ ]:
from preprocessing.calc_sephi import get_sephi_RM17

In [38]:
sephi = get_sephi_RM17(pl_mass, pl_rad, pl_a, teff, lum, st_mass, age, verbose=False)

In [40]:
print(sephi)
print(sephi.shape)

[nan nan nan ... nan nan nan]
(4734,)


In [ ]:
# TODO: move the sephi functions back to sephi file
# TODO: add the sephi to the data frame
# TODO: Use the 'best' stellar parameters to calc sephi